# Indigo v2.1 Algo Interest Data Generation and Ideation

##### Indigo Protocol is a synthetics based protocol on Cardano. Users can mint synthetic assets like iBTC/iETH/iUSD by providing ADA as collateral.In Indigo v2, the interest rate concept was introduced and v2.1 onwards an algorithmic interest was introduced, which varies according to the state of the protocol iAssets. The visualisations of how the interest rate would vary is shown in this notebook using charts and graph. Thi visual data was presented to the community as a part of the DAO proposal for Indigo v2.1.

**This notebook discussed the steps involved in the generation of data for the algo interest dynamic model(iUSD_algo_model.xlsx) using various internal and external API's. This model creates dynamic model showcasing the interest rate trend based on the historical protocol data. This model was an integral part of the alog interest modelling process and was used extensively by the Indigo team to finalize algo interest parameters for all the iAssets.**

***Step1:Generating list of timestamps***

In [11]:
import time
import time, calendar
from datetime import date
import os
import pandas as pd
import requests
import urllib.request
import json
from datetime import datetime
from typing import Final
import glob

In [12]:

start_time= 1725042600
delay=86400
i=1
timestamps=[start_time]

for i in range (1,6):
    time=start_time+delay
    timestamps.append(time)
    start_time=time
    i=i+1
timestamps

[1725042600, 1725129000, 1725215400, 1725301800, 1725388200, 1725474600]

***Step2: Getting the historical iAsset prices based using CoinGecko API and filtering it based on the selected dates***

In [13]:
import time, calendar
from datetime import date
import os
import pandas as pd
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

iUSD_price=[]
iBTC_price=[]
iETH_price=[]
ADA_price=[]

########ADA
data1=cg.get_coin_market_chart_by_id(id='cardano', vs_currency='usd', days=364)
data1.pop('market_caps')
data1.pop('total_volumes')
dates1=[]
price1=[]
for i,j in data1["prices"]:
    new1=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    block=new1.split(" ")[0]
    if pd.to_datetime(block)==date.today():
        break
    if block in dates1:
        break
    else:
        dates1.append(block)
    price1.append(j)


dictionary1= {'Date':dates1,'INDY Price':price1}


########BTC
data2=cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=364)
data2.pop('market_caps')
data2.pop('total_volumes')
dates2=[]
price2=[]
for i,j in data2["prices"]:
    new2=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    dates2.append(new2.split(" ")[0])
    price2.append(j)

dictionary2= {'Date':dates2,'BTC Price':price2}
# dictionary2

########ETH
data3=cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='usd', days=364)
data3.pop('market_caps')
data3.pop('total_volumes')
dates3=[]
price3=[]
for i,j in data3["prices"]:
    new3=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    dates3.append(new3.split(" ")[0])
    price3.append(j)
dictionary3= {'Date':dates3,'ETH Price':price3}
# dictionary3

########iUSD
data4=cg.get_coin_market_chart_by_id(id='tether', vs_currency='usd', days=364)
# pd.read_json(data)
data4.pop('market_caps')
data4.pop('total_volumes')
dates4=[]
price4=[]
for i,j in data4["prices"]:
    new4=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    date=new4.split(" ")[0]
    if date in dates4:
        continue
    dates4.append(date)
    price4.append(j)
dictionary4= {'Date':dates4,'BTC Price':price4}

##################logic_BTC
btc_price=[]
for i in dictionary1["Date"]:
#     print(i)
    for j in set(dates2):
        if i==j:
            btc_price.append(price2[dates2.index(j)])
# len(btc_price)
# ##################logic_ETH
eth_price=[]
for i in dictionary1["Date"]:
    for j in set(dates3):
        if i==j:
            eth_price.append(price3[dates3.index(j)])
# eth_price
# ##################logic_USDT
usd_price=[]
for i in dictionary1["Date"]:
    for j in set(dates4):
        if i==j:
            usd_price.append(price4[dates4.index(j)])
# len(usd_price)


dictionary_combo= {'Date':dates1,'ADA_Price':price1, 'iBTC_Price':btc_price, 'iETH_Price':eth_price, 'iUSD_Price':usd_price}
df1 = pd.DataFrame(dictionary_combo)
# print(df1.to_string())

##Selection of dates
start_date = '2024-08-31'
end_date = '2024-09-05'

# Filtering the DataFrame
filtered_df = df1[(df1['Date'] >= start_date) & (df1['Date'] <= end_date)]
filtered_df = filtered_df.round(6)
print(filtered_df)
ADA_price = filtered_df['ADA_Price'].tolist()
iUSD_price = filtered_df['iUSD_Price'].tolist()
iETH_price = filtered_df['iETH_Price'].tolist()
iBTC_price = filtered_df['iBTC_Price'].tolist()

print(f'Timetamp: {timestamps}')
print(f'iUSD: {iUSD_price}')
print(f'iBTC: {iBTC_price}')
print(f'iETH: {iETH_price}')
print(f'ADA: {ADA_price}')

ConnectionError: HTTPSConnectionPool(host='api.coingecko.com', port=443): Max retries exceeded with url: /api/v3/coins/cardano/market_chart?vs_currency=usd&days=364 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000188989DB320>: Failed to resolve 'api.coingecko.com' ([Errno 11001] getaddrinfo failed)"))

***Step 3: Calculating the interest accrued, adjusted collateral and corresponding CR***

In [ ]:
%%time

#interest rate details
interest_rate = 20.075
interest_rate_introduced=1714320140000

#defining lists
inner=[]
collateral_iUSD_amount=[]
minted_iUSD_amount=[]
int_iUSD=[]
adjusted_collateral_iUSD_amount=[]
adjusted_int_iusd=[]

iAsset_price=iUSD_price

#calculated accrued interest and the corresponding CR
for i, j, k in zip(timestamps, ADA_price, iAsset_price):
    headers = {
        # Already added when you pass json=
        # 'content-type': 'application/json',
    }

    json_data = {
        'timestamp': i,
    }

    response = requests.post('https://analytics.indigoprotocol.io/api/cdps', headers=headers, json=json_data)
    ans=response.json()
    df3 = pd.DataFrame(ans)
    columns_to_drop = ['output_hash','output_index','owner']
    df3 = df3.drop(columns=columns_to_drop)


    df3['collateralAmount']= df3['collateralAmount']/1000000
    df3['mintedAmount']= df3['mintedAmount']/1000000
    df3['interest_iasset_amount']= df3['interest_iasset_amount']/1000000


    df3['interest_last_updated'] = df3.apply(lambda x: interest_rate_introduced if x['interest_last_updated'] < interest_rate_introduced else x['interest_last_updated'], axis=1)
    df3['interest_time'] = (i * 1000) - df3['interest_last_updated']
    df3.insert(6, column="adjustedinterestAmount", value=(df3['interest_iasset_amount']+((df3['interest_time'] / (1000 * 60 * 60 * 24 * 365)) * (interest_rate / 100) * df3['mintedAmount'])))
    df3.insert(3, column="adjustedcollateralAmount", value=((df3['collateralAmount'])-(df3['adjustedinterestAmount']*k/j)))
    df3.insert(7, column="CR", value=((df3['adjustedcollateralAmount']*j)/(df3['mintedAmount']*k)))


    df3=df3.groupby(['asset'])
    df4=df3.sum()

    value_iUSD_mintedAmount = df4.loc['iUSD', 'mintedAmount']
    minted_iUSD_amount.append(value_iUSD_mintedAmount)

    value_iUSD_collateralAmount = df4.loc['iUSD', 'collateralAmount']
    collateral_iUSD_amount.append(value_iUSD_collateralAmount)

    value_iUSD_adjustedcollateralAmount = df4.loc['iUSD', 'adjustedcollateralAmount']
    adjusted_collateral_iUSD_amount.append(value_iUSD_adjustedcollateralAmount)

    value_iUSD_int = df4.loc['iUSD', 'interest_iasset_amount']
    int_iUSD.append(value_iUSD_int)

    value_iUSD_int_adjusted= df4.loc['iUSD', 'adjustedinterestAmount']
    adjusted_int_iusd.append(value_iUSD_int_adjusted)

print("ADA collateral"+ f': {collateral_iUSD_amount}')
print('=======================================================================')
print(":iUSD minted amount" + f': {minted_iUSD_amount}')
print('=======================================================================')
print("ADA collateral(post interest)" + f': {adjusted_collateral_iUSD_amount}')
print('=======================================================================')
print("Interest Accrued"+f': {adjusted_int_iusd}')
print('=======================================================================')




***Step 4: Create the final dataframe and export as Excel/csv***

In [ ]:
#Create a dictionary
data_iUSD = {'timestamp': timestamps, 'iUSD_price': iUSD_price, 'ADA_price': ADA_price, 'iUSD_collateral_amount': adjusted_collateral_iUSD_amount, 'iUSD_minted_amount': minted_iUSD_amount}

# Create a new DataFrame
df_iUSD = pd.DataFrame(data_iUSD)
df_iUSD['iUSD_price'] = df_iUSD['iUSD_price'].astype(float)
df_iUSD['ADA_price'] = df_iUSD['ADA_price'].astype(float)
df_iUSD['iTCR'] = (df_iUSD['iUSD_collateral_amount']*df_iUSD['ADA_price'])/(df_iUSD['iUSD_minted_amount'])

# Print the DataFrame
print(df_iUSD.to_string())

#Export to excel
# df_iUSD.to_excel('algo_int_output_iUSD_latest.xlsx', index=False)

**This final data has timestamp, iUSD_price, ADA_price, iUSD_collateral_amount, iUSD_minted_amount, iTCR. These are some of historical input parameters for the algo interest model**